## 2. Cruzamento dos Dados (Enunciado Questões + Microdados)

* **Input**: Local do CSV de Questões Extraídas + Local do CSV de Microdados

* **Output**: CSV com as colunas -> *numero_questao*, *enunciado*, *alternativas*, *nu_param_B* e *gabarito*

In [1]:
import pandas as pd

In [31]:
# Definição de Variáveis de Input e Output
year = "2021"

question_path = f"../data/extracted_questions/enem_{year}.csv"
microdados_path = f"../data/microdados/microdados_{year}.csv"
output_path = f"../data/merged_data/enem_{year}.csv"

In [32]:
# Ler o CSV extraído do PDF (resultado de Text_Extraction.ipynb)
df_questions = pd.read_csv(question_path, encoding="utf-8", quotechar='"')

# Ler o CSV de microdados
df_microdados = pd.read_csv(microdados_path, delimiter=";", encoding="latin1")

In [33]:
# Filtrando os microdados para SG_AREA 'CH', TX_COR 'AZUL' e CO_PROVA 'X'
# ATENÇÃO: O Código da Prova para a Aplicação Regular muda a cada ano
df_microdados_filtrado = df_microdados[
    (df_microdados["SG_AREA"] == "CH")  # Ciência Humanas
    & (df_microdados["TX_COR"].str.upper() == "AZUL")  # Apenas caderno azul
]

# Determinar o menor valor de CO_PROVA encontrado
menor_co_prova = int(df_microdados_filtrado["CO_PROVA"].min())

# Filtrar novamente considerando apenas o menor CO_PROVA
df_microdados_filtrado = df_microdados_filtrado[
    df_microdados_filtrado["CO_PROVA"] == menor_co_prova
]

# Selecionar apenas as colunas de interesse dos microdados
df_microdados_sel = df_microdados_filtrado[
    ["CO_POSICAO", "NU_PARAM_B", "TX_GABARITO"]
].copy()

In [34]:
# Certificar que os tipos das chaves de junção são compatíveis
df_questions["numero_questao"] = df_questions["numero_questao"].astype(str)
df_microdados_sel["CO_POSICAO"] = df_microdados_sel["CO_POSICAO"].astype(str)

# Fazer o cruzamento usando o número da questão (numero_questao e CO_POSICAO)
df_merged = pd.merge(
    df_questions,
    df_microdados_sel,
    left_on="numero_questao",
    right_on="CO_POSICAO",
    how="left",
)

In [35]:
# Remover a coluna CO_POSICAO (redundante)
df_merged.drop(columns=["CO_POSICAO"], inplace=True)

# Renomeando colunas
df_merged.rename(columns={
    "NU_PARAM_B": "nu_param_B",
    "TX_GABARITO": "gabarito"
}, inplace=True)


# Salvando DF resultante
df_merged.to_csv(output_path, index=False, encoding="utf-8")

print("Merge concluído.")

Merge concluído.


In [36]:
df = pd.read_csv(output_path)
df.head()

,numero_questao,enunciado,alternativas,nu_param_B,gabarito
0,46,"Seu turno de trabalho acabou, você já está em ...",A: proteção da vida privada.; B: ampliação de ...,0.59033,B
1,47,Quando a taxa de remuneração do capital excede...,A: ideologia do mérito.; B: direito de nascime...,1.33796,A
2,48,"Atualmente, o Programa de Melhoramento ""Uvas d...",A: espécies nativas ameaçadas.; B: cadeias eco...,1.61961,D
3,49,TEXTO I\nPortadoras de mensagem espiritual do ...,A: ampliação do comércio de imagens sacras.; B...,0.38656,C
4,50,"TEXTO I\nEm 2016, foram gerados 44,7 milhões d...",A: Exercício pleno da cidadania.; B: Divisão i...,1.67763,B
